#Markov Chains for Rhythm 

We will proceed much in the way we did with the melodic sequences. Let's go!

In [21]:
import pickle

In [22]:
class Markov(object):
    """A container for holding Markov chains of various orders, 
    where the events in the given 'current state' may occur temporally 
    before and/or after the note to be examined as the 'next state'.
    """
    
    def __init__(self, before = 0, after = 0, mode = 0):
        """Initialize a Markov object with `before` notes before the
        space to be filled, `after` notes after the space to be filled,
        and with `mode` equal to 0, 1 or 2 according to whether the 'current
        state' contains notes only before, only after, or both before and after
        the note to be filled in the 'next state'.
        
        Inputs: 
        before - int - number of notes before next state
        after - int - number of notes after next state
        mode - int in range(3) - mode of the chain(see above)
        
        Outputs - Markov object
        """
        
        if mode == 0:
            assert before != 0 and after == 0
        elif mode == 1:
            assert before == 0 and after != 0
        else:
            assert before != 0 and after != 0

        self.before = before
        self.after = after
        self.mode = mode
        self.state_dict = {}
        
    def add_data(self, seq, result):
        """Add one (current state -> next state) instance to the dictionary.
        Store each instance as a tally, to be normalized later.
        
        Inputs:
        seq - if mode = 0, seq is a tuple of length before
              if mode = 1, seq is a tuple of length after
              if mode = 2, seq is a list of length two, with first element
                           a tuple of length before and with second element
                           a tuple of length after
        result - int - single event
        
        Outputs:
        None
        """
        
        if self.mode == 0:
            assert isinstance(seq, tuple) and len(seq) == self.before
        elif self.mode == 1:
            assert isinstance(seq, tuple) and len(seq) == self.after
        else:
            assert (isinstance(seq, tuple) and len(seq) == 2 and
                    isinstance(seq[0], tuple) and len(seq[0]) == self.before and
                    isinstance(seq[1], tuple) and len(seq[1]) == self.after)
            
        if seq not in self.state_dict:
            self.state_dict[seq] = {result: 1}
        elif result in self.state_dict[seq]:
            self.state_dict[seq][result] += 1
        else:
            self.state_dict[seq][result] = 1
            
    def normalize(self):
        """Convert the state_dict dictionary from counts to probabilities.
        
        Inputs: None
        
        Outputs: None
        """
        
        for seq in self.state_dict:
            sum = 0
            for result in self.state_dict[seq]:
                sum += self.state_dict[seq][result]
            for result in self.state_dict[seq]:
                self.state_dict[seq][result] /= float(sum)

Since we've gotten rid of all concurrent rhythmic tick events, we won't have to deal with breaking up chords like we did with the melodic sequences. However, since we want the rhythmic events to be time-invariant (i.e. it doesn't matter WHEN in the file they occur), we will 'recenter' all of the data as we did with the longer melodic chains.

In [33]:
def iterate_rhythm(rhy, mark):
    """Adds one new observation to the Markov chain training process.
    
    Inputs: 
    mel - a list of lists representing a rhythm sequence
    mark - a Markov object
    
    Outputs:
    None (instead adds data to markov object directly)
    """
    before, after = mark.before, mark.after
    full_length = before + after + 1
    
    for i in range(len(rhy) - full_length):
        seq = rhy[i:i + full_length]
        before_seq, after_seq, val = seq[:before], seq[-after:], seq[before]
        before_seq, after_seq, val = recenter(before_seq, after_seq, val)
        mode = mark.mode
        if mode == 0:
            mark.add_data(before_seq, val)
        elif mode == 1:
            mark.add_data(after_seq, val)
        else:
            mark.add_data((before_seq, after_seq), val)
            
def recenter(before, after, val):
    """Recenters sequence(s) so that the first note valued encountered is 0 and every other
    note value is relative to the first.
    
    Ex: before = [45, 47], after = [44, 45], val = 40 returns ([0, 2], [-1, 0], -5)
        before = [], after = [100, 100, 99], val = 100 returns ([], [0, 0, -1], 0)
        
    Inputs: before and after are lists of note values with combined length at least 3,
            val is a note value
    
    Outputs: Tuple of length 3 of the scaled before, after, and val note values.
    """
    
    if len(before) == 0:
        to_subtract = after[0]
    else:
        to_subtract = before[0]
    return (tuple(map(lambda x: round(x - to_subtract, 4), before)), 
            tuple(map(lambda x: round(x - to_subtract, 4), after)), 
            round(val - to_subtract, 4))

Let's try building a smaller-order chain on the level 1 quantized rhythm sequences as a test.

In [24]:
with open('../all_rhythms_level_1.pkl', 'r') as f:
    all_rhythms_level_1 = pickle.load(f)

In [ ]:
mark = Markov(1, 1, 2)
for k in range(len(all_rhythms_level_1)):
    iterate_rhythm(all_rhythms_level_1[k], mark)
mark.normalize()
with open('../markov_models/markov_rhythm_level_1_112.pkl', 'w') as f:
    pickle.dump(mark, f)

In [31]:
print mark.state_dict[(0.0,), (1.0,)]

{0.75: 0.030659787604253117, 0.5: 0.8656840672853778, 0.8333: 0.0009844773277688334, 0.25: 0.02448856655622897, 0.4167: 0.003231656977185117, 0.5833: 0.0033430336491894856, 0.3334: 0.00037443363662168985, 0.6667: 0.03662082310394217, 0.3333: 0.03249945286751593, 0.6666: 0.0006016651606980471, 0.9167: 0.00025641192322665876, 0.0833: 0.000320117646124099, 0.1667: 0.0009355062618680801}


Awesome! This is MUCH more compact than the alternative (912 KB for this one pickle file).

On to making each of the models!

In [ ]:
for before in range(4):
    for after in range(4):
        if before + after > 4:
            continue
        if before == 0:
            if after == 0:
                continue
            mode = 1
        elif after == 0:
            mode = 0
        else:
            mode = 2
        mark = Markov(before, after, mode)
        for k in range(len(all_rhythms_level_1)):
            iterate_rhythm(all_rhythms_level_1[k], mark)
        mark.normalize()
        with open('../markov_models/markov_rhythm_level_1_' + str(before) + str(after) + str(mode) + '.pkl', 'w') as f:
            pickle.dump(mark, f)

In [ ]:
for before in range(4):
    for after in range(4):
        if before + after > 4:
            continue
        if before == 0:
            if after == 0:
                continue
            mode = 1
        elif after == 0:
            mode = 0
        else:
            mode = 2
        mark = Markov(before, after, mode)
        for k in range(len(all_rhythms_level_2)):
            iterate_rhythm(all_rhythms_level_2[k], mark)
            if k % 1000 == 0:
                print k
        mark.normalize()
        with open('../markov_models/markov_rhythm_level_2_' + str(before) + str(after) + str(mode) + '.pkl', 'w') as f:
            pickle.dump(mark, f)
        print before, after

In [ ]:
for before in range(4):
    for after in range(4):
        if before + after > 4:
            continue
        if before == 0:
            if after == 0:
                continue
            mode = 1
        elif after == 0:
            mode = 0
        else:
            mode = 2
        mark = Markov(before, after, mode)
        for k in range(len(all_rhythms_level_3)):
            iterate_rhythm(all_rhythms_level_3[k], mark)
            if k % 1000 == 0:
                print k
        mark.normalize()
        with open('../markov_models/markov_rhythm_level_3_' + str(before) + str(after) + str(mode) + '.pkl', 'w') as f:
            pickle.dump(mark, f)
        print before, after

We now have three sets of Markov Chains at different levels of granularity.